<a href="https://colab.research.google.com/github/felixzhao/AIML425-ASSN-4/blob/main/notebooks/V1-FZ-AIML425_Assn_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import plotly.express as px

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 1. Generate 3D data

In [18]:
def generate_points_on_cube_surface(n_points_per_face):
    """
    Generate points uniformly distributed over the surface of a cube.

    Parameters:
    - n_points_per_face: Number of points to generate on each face of the cube.

    Returns:
    - points: (3, 6*n_points_per_face) array of points on the cube surface.
    """
    faces = []
    for _ in range(6):
        face = np.random.rand(3, n_points_per_face)
        axis = _ // 2  # 0: x, 1: y, 2: z
        face[axis, :] = (_ % 2)  # 0 or 1, depending on the face
        faces.append(face)
    return np.hstack(faces)

n_points_per_face = 1000
points = generate_points_on_cube_surface(n_points_per_face)

fig = px.scatter_3d(x=points[0], y=points[1], z=points[2], opacity=0.5)
fig.show()


# 2. Create a dataset and dataloaders

In [12]:
points = points.T  # Transpose to have shape (n_samples, 3)

# 2. Create a dataset and dataloaders
dataset = TensorDataset(torch.tensor(points, dtype=torch.float32))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 3. Define and train the autoencoder

In [14]:
# 3. Define and train the autoencoder
class Autoencoder(nn.Module):
    def __init__(self, bottleneck_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Linear(128, bottleneck_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

bottleneck_dim = 2
model = Autoencoder(bottleneck_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for data, in dataloader:
        outputs = model(data)
        loss = criterion(outputs, data)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.0416
Epoch [2/10], Loss: 0.0400
Epoch [3/10], Loss: 0.0421
Epoch [4/10], Loss: 0.0133
Epoch [5/10], Loss: 0.0122
Epoch [6/10], Loss: 0.0316
Epoch [7/10], Loss: 0.0061
Epoch [8/10], Loss: 0.0053
Epoch [9/10], Loss: 0.0196
Epoch [10/10], Loss: 0.0053


# 4. Evaluate the autoencoder

In [15]:
model.eval()
with torch.no_grad():
    reconstructed = model(torch.tensor(points, dtype=torch.float32)).numpy()


## Visualize original and reconstructed points

In [19]:
# fig = px.scatter_3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], opacity=0.5, title="Original Points")
# fig.show()

In [17]:
fig = px.scatter_3d(x=reconstructed[:, 0], y=reconstructed[:, 1], z=reconstructed[:, 2], opacity=0.5, title="Reconstructed Points")
fig.show()